In [9]:
import csv

with open('input_file_wenhu.csv', mode='w') as fw:
    with open('input_file_shiyang.csv', mode='w') as fs:
        with open('input_file_yunkai.csv', mode='w') as fy:
            
            #output_wenhu = output[:5000]
            #fileds = ['url1', 'q1', 'a1', 'url2', 'q2', 'a2', 'url3', 'q3', 'a3', 'url4', 'q4', 'a4', 'url5', 'q5', 'a5']
            #writer = csv.DictWriter(fw, fieldnames=fileds)
            #writer.writeheader()
            #for i in range(0, len(output_wenhu) - 5, 5):
            #    writer.writerow({'url1':output_wenhu[i]['url'], 'q1':output_wenhu[i]['q'], 'a1':output_wenhu[i]['a'],
            #                    'url2':output_wenhu[i+1]['url'], 'q2':output_wenhu[i+1]['q'], 'a2':output_wenhu[i+1]['a'],
            #                    'url3':output_wenhu[i+2]['url'], 'q3':output_wenhu[i+2]['q'], 'a3':output_wenhu[i+2]['a'],
            #                    'url4':output_wenhu[i+3]['url'], 'q4':output_wenhu[i+3]['q'], 'a4':output_wenhu[i+3]['a'],
            #                    'url5':output_wenhu[i+4]['url'], 'q5':output_wenhu[i+4]['q'], 'a5':output_wenhu[i+4]['a']})
            
            num = 10
            
            output_shiyang = output[5000:20000]
            fields = []
            for i in range(1, num+1):
                fields.extend(['url{}'.format(i), 'q{}'.format(i), 'a{}'.format(i)])
            writer = csv.DictWriter(fs, fieldnames=fields)
            writer.writeheader()
            for i in range(0, len(output_shiyang) - num, num):
                elem = {}
                for j in range(1, num+1):
                    elem['url{}'.format(j)] = output_shiyang[i+j-1]['url']
                    elem['q{}'.format(j)] = output_shiyang[i+j-1]['q']
                    elem['a{}'.format(j)] = output_shiyang[i+j-1]['a']
                
                writer.writerow(elem) 
            
            
            output_yunkai = output[20000:]
            fields = []
            for i in range(1, num+1):
                fields.extend(['url{}'.format(i), 'q{}'.format(i), 'a{}'.format(i)])
            writer = csv.DictWriter(fy, fieldnames=fields)
            writer.writeheader()
            for i in range(0, len(output_yunkai) - num, num):
                elem = {}
                for j in range(1, num+1):
                    elem['url{}'.format(j)] = output_yunkai[i+j-1]['url']
                    elem['q{}'.format(j)] = output_yunkai[i+j-1]['q']
                    elem['a{}'.format(j)] = output_yunkai[i+j-1]['a']
                
                writer.writerow(elem) 
                

In [52]:
def regex(inp):
    inp = re.sub(r"> +", r">", inp)
    inp = re.sub(r" +<", r"<", inp)
    inp = re.sub(r">@", r">", inp)
    inp = re.sub(r"\*", r"", inp)
    inp = re.sub(r"#", r"", inp)
    inp = re.sub(r"‡", r"", inp)
    #inp = re.sub(r".0([^0-9])", r"\1", inp)
    for month, abb in [("January", "Jan"), ("February", "Feb"), ("March", "March"), ("April", "April"), 
                   ("May", "May"), ("June","June") , ("July", "July"), ("August", "Aug"), 
                   ("September", "Sep"), ("October", "Oct"), ("November", "Nov"), ("December", "Dec")]:
        inp = re.sub(r"({})([0-9]+)".format(month), r"\1 \2", inp)
        inp = re.sub(r"({})([0-9]+)".format(abb), r"\1 \2", inp)
    return inp

In [54]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')
import jsonlines
import csv
import re
"""
with open('all_html_original.csv', 'w') as fw:
    fields = ['url', 'content']
    writer = csv.DictWriter(fw, fieldnames=fields)
    writer.writeheader()
"""
with open('WikiTableQuestions/wikidata/train.tables.jsonl') as f:
    data = jsonlines.Reader(f)
    for d in data:
        f = '<table class="wikitable"><tr>'
        for h in d['header']:
            f += "<th> {} </th>".format(h)
        f += "</tr>"
        for r in d['rows']:
            f += "<tr>"
            for elem in r:
                f += "<td> {} </td>".format(elem)
            f += "</tr>"
        f += "</table>"
        #writer.writerow({'url': d['id'], "content": f})
        with open('WikiTableQuestions/wikidata/all_html/{}.html'.format(d['id']), 'w') as fw:
            print >> fw, regex(f)

In [22]:
import re

def regex_equation(line):
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=", r"\1\2+\3+\4+\5=", line)
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)=", r"\1\2+\3+\4=", line)
    line = re.sub(r"([^+-])([0-9]+)-([0-9]+)=", r"\1\2+\3=", line)
    #line = re.sub(r"([^+-])([0-9]+)-([0-9]+)+([0-9]+)=", r"\1\2+\3+\4=", line)
    #line = re.sub(r"([^+-])([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=", r"\1+\2+\3+\4=", line)    
    return line

In [24]:
regex_equation("67-54-69-68=270)")

'67+54+69+68=270)'

In [26]:
with open('/tmp/list.txt') as f:
    for line in f:
        with open('WikiTableQuestions/wikidata/all_html/' + line.strip(), 'r') as f1:
            content = f1.readline().strip()
        with open('WikiTableQuestions/wikidata/all_html/' + line.strip(), 'w') as f1:
            print >> f1, regex_equation(content)
        #print regex_equation(content)

In [81]:
import json

with open('WikiTableQuestions/wikidata/train_gold.json') as f:
    raw_output = json.load(f)
    output = ["/".join([str(__) for __ in _]) for _ in raw_output]

with open('WikiTableQuestions/wikidata/all_training.tsv', 'w') as fw:
    print >> fw, "id\tutterance\tcontext\ttargetValue"
    with open('WikiTableQuestions/wikidata/train.jsonl') as f:
        data = jsonlines.Reader(f)
        idx = 1
        for d, o in zip(data, output):
            print >> fw, "{}\t{}\t{}\t{}".format(idx, d['question'].replace('\t', ''), d['table_id'], o)
            idx += 1

In [2]:
import os
import pandas

files = pandas.read_table('WikiTableQuestions/wikidata/all_training.tsv', delimiter="\t")

output = []
for f in os.listdir('WikiTableQuestions/wikidata/all_html/'):
    if "html" in f:
        numbering = f.split('.')[0]
        results = files[files.context == numbering]
        
        for q, a in zip(results.utterance, results.targetValue):
            tmp = {}
            tmp["url"] = 'https://raw.githubusercontent.com/wenhuchen/Interface/master/WikiTableQuestions/wikidata/all_html/{}.html'.format(numbering)
            tmp["q"] = q
            tmp["a"] = a
            output.append(tmp)

In [23]:
with open('WikiTableQuestions/data/all_training_new.tsv', 'w') as fw:
    with open('WikiTableQuestions/data/all_training.tsv') as f:
        for line in f:
            if "csv" in line:
                try:
                    t1, t2, t3, t4 = line.strip().split('\t')
                except Exception:
                    print line.strip()
                    continue
                    
                try:
                    f1, f2, f3 = t3.split('/')
                except Exception:
                    print t3.strip()
                    continue
                    
                t3 = f2.split('-')[0] + "-" + f3
                new_line = "\t".join([t1, t2, t3, t4])
                print >> fw, new_line
            else:
                print >> fw, line.strip()

ns-41   what film did ishaan dev give a song to, immediately after "ringtone" in 2010?  csv/204-csv/546.csv "Thriller"


In [18]:
import pandas

result = pandas.read_table('WikiTableQuestions/wikidata/all_training.tsv')

new_result = result[~result.targetValue.isin(['None', 'n/a', ])]

new_result.to_csv('WikiTableQuestions/wikidata/all_training_new.tsv', sep='\t')

In [13]:
for r in result['Answer.d1']:
    print r

She's the Boss
as of october 25, 2005, there are 9717 active voters.
An example of a transcription factor is myc gene.
2011
Yes
Yes, Tripoli is still considered a municipality in Arcadia since its 2011 reformation.
A character named William Armstrong appearred in three of the series.
Ron Turcotte rode secretariat in 1973.
Pier Ruggero Piccio's term started 1 January 1926.
BBC Local Radio cost the most to run in 2012/13.
The combined area size of the yamato flat inland plain and the yamato highland is 1344.16.
The average high temperature in Fayetteville, Arkansas in June is 83.5 degrees Farenheit.
Earl Ball coached Muncie Flyers for 6 years.
Susana Martinez is the governor of New Mexico.
Wanamaker was nominated for best actress in a revival for Twelfth Night in 1984.
As of October 25, 2005, the Democratic party had 7,115 inactive voters.
1
9 months have an average low below 1 degree celsius.
William Friedkin won an academy award for best director in the film, "the french connection."?


In [31]:
import json
import csv
import pandas

result = pandas.read_csv('results_v2.csv')
filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower():
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

with open('v2_results.json', 'w') as f:
    json.dump(items, f, indent=2)
    
with open('v2_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = items[i + j][0]
            elem["s{}".format(j + 1)] = items[i + j][3]
            
        writer.writerow(elem)

In [32]:
import json
import pandas

with open('v2_results.json', 'r') as f:
    data = json.load(f)

keys = [_[1] for _ in data]
result = pandas.read_csv('results_v2_new.csv')
filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
        
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower() and r['Input.q{}'.format(i)] not in keys:
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [33]:
import csv

result = pandas.read_csv('results_v3.csv')

filtered = result[result.AssignmentStatus=="Approved"]

new_items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num + 1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in str(r['Answer.d{}'.format(i)]).lower():
            new_items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

new_items = new_items + items

with open('v3_results.json', 'w') as f:
    json.dump(new_items, f, indent=2)
    
num = 5
with open('v3_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(new_items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = new_items[i + j][0]
            elem["s{}".format(j + 1)] = new_items[i + j][3]
            
        writer.writerow(elem)

In [34]:
print len(new_items)

14315


In [4]:
s = ""
for month, abb in [("January", "Jan"), ("February", "Feb"), ("March", "March"), ("April", "April"), 
                   ("May", "May"), ("June","June") , ("July", "July"), ("August", "Aug"), 
                   ("September", "Sep"), ("October", "Oct"), ("November", "Nov"), ("December", "Dec")]:
    s += ";s/({})([1-9]+)/\\1 \\2/g;s/({}),([1-9]+)//g".format(month, abb)
print s

;s/(January)([1-9]+)/\1 \2/g;s/(Jan),([1-9]+)//g;s/(February)([1-9]+)/\1 \2/g;s/(Feb),([1-9]+)//g;s/(March)([1-9]+)/\1 \2/g;s/(March),([1-9]+)//g;s/(April)([1-9]+)/\1 \2/g;s/(April),([1-9]+)//g;s/(May)([1-9]+)/\1 \2/g;s/(May),([1-9]+)//g;s/(June)([1-9]+)/\1 \2/g;s/(June),([1-9]+)//g;s/(July)([1-9]+)/\1 \2/g;s/(July),([1-9]+)//g;s/(August)([1-9]+)/\1 \2/g;s/(Aug),([1-9]+)//g;s/(September)([1-9]+)/\1 \2/g;s/(Sep),([1-9]+)//g;s/(October)([1-9]+)/\1 \2/g;s/(Oct),([1-9]+)//g;s/(November)([1-9]+)/\1 \2/g;s/(Nov),([1-9]+)//g;s/(December)([1-9]+)/\1 \2/g;s/(Dec),([1-9]+)//g


In [61]:
import pandas
import simplejson

result = pandas.read_csv('v2_refine.csv')
filtered_result = result[result.AssignmentStatus=="Approved"]

items = []
num = 10
for i, r in filtered_result.iterrows():
    for i in range(1, num+1):
        items.append((r["Input.url{}".format(i)],"-" ,"-" , r["Answer.d{}".format(i)]))

with open('v2_rewrite.json', 'w') as f:    
    simplejson.dump(items, f, encoding='utf-8', ignore_nan=True)

In [62]:
import os
#import pandas
import re
import csv

def dealwithNum(inp):
    inp = re.sub(r"([^0-9.])(0)([0-9])%", r"\1\2.\3%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])%", r"\1\2\3.\4%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])%", r"\1\2.\3\4%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])([0-9])%", r"\1\2\3.\4\5%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])([0-9])%", r"\1\2.\3\4\5%", inp)
    
    inp = re.sub(r"10000%", r"100.00%", inp)
    inp = re.sub(r"1000%", r"100.0%", inp)
    
    inp = re.sub(r"([^0-9.])([1-9])([0-9])([0-9])([0-9])([0-9])%", r"\1\2\3.\4\5\6%", inp)
    inp = re.sub(r"([^0-9.])(0)([0-9])([0-9])([0-9])([0-9])%", r"\1\2.\3\4\5\6%", inp)
    
    inp = re.sub(r",([0-9])%", r".\1%", inp)
    inp = re.sub(r",([0-9])([0-9])%", r".\1\2%", inp)
    inp = re.sub(r",([0-9])([0-9])([0-9])%", r".\1\2\3%", inp)
    
    
    #inp = re.sub(r"([0-9]),([0-9])", r"\1\2", inp)
    return inp


with open('all_html.csv', 'w') as f:
    fields = ["url", "content"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for f in os.listdir('WikiTableQuestions/wikidata/all_html/'):
        with open('WikiTableQuestions/wikidata/all_html/' + f, 'r') as fr:
            string = fr.readline().strip()
        
        writer.writerow({"url": f, "content": dealwithNum(string)})

#inp_s = "<td>123% <td> 0123% <td>012% <td>1233%  <td>12333% <td>10000% <td>01333%"
#dealwithNum(inp_s)

In [64]:
import pandas

table = pandas.read_csv('all_html.csv')

for i, item in table.iterrows():
    name = item['url']
    content = item['content']
    with open('WikiTableQuestions/wikidata/all_html/{}'.format(name), 'w') as f:
        print >> f, content

In [36]:
import json
import csv
import pandas

result = pandas.read_csv('results_v4_1.csv')

filtered = result[result.AssignmentStatus=="Approved"]

items = []
for i, r in filtered.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for i in range(1, num+1):
        if r['Input.a{}'.format(i)] not in ['None', 'n/a'] and " par " not in r['Answer.d{}'.format(i)].lower():
            items.append((r['Input.url{}'.format(i)], r['Input.q{}'.format(i)], r['Input.a{}'.format(i)], r['Answer.d{}'.format(i)]))

with open('v4_1_results.json', 'w') as f:
    json.dump(items, f, indent=2)

num = 5
with open('v4_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
    
    #fields.extend(["url11", "s11"])
    
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(items) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = items[i + j][0]
            elem["s{}".format(j + 1)] = items[i + j][3]
        
        #elem['url11'] = "https://raw.githubusercontent.com/wenhuchen/Interface/master/WikiTableQuestions/wikidata/all_html/2-1236260-1.html"
        #elem['s11'] = "Total is the rank of total"
            
        writer.writerow(elem)

In [56]:
import pandas
import csv
import json
import simplejson

r1 = pandas.read_csv('partial_refine_v2.csv')
r2 = pandas.read_csv('partial_refine_v3.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

results = {}
finished = []
for i, r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}
            results[html_name]['text'].append(t)
            results[html_name]['label'].append(1)
        finished.append(r['Input.s{}'.format(j)])
        
for i, r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(1)
        finished.append(r['Input.s{}'.format(j)])

r1 = pandas.read_csv('partial_neg_v2.csv')
r2 = pandas.read_csv('partial_neg_v3.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

for i, r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            html_name = r['Input.url{}'.format(j)].split('/')[-1]
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(-1)

for i, r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        t = r['Answer.d{}'.format(j)]
        if t and isinstance(t, str) and t.lower() not in ["na", "none", "n/a", "no"]:
            html_name = r['Input.url{}'.format(j)].split('/')[-1]
            if html_name not in results:
                results[html_name] = {"text": [], "label": []}        
            results[html_name]['text'].append(r['Answer.d{}'.format(j)])
            results[html_name]['label'].append(-1)
            
with open('READY/prelim.json', 'w') as f:
    simplejson.dump(results, f, encoding='utf-8', ignore_nan=True)

In [ ]:
# Prepare results
inp_v2 = pandas.read_csv('v2_rewrite_input.csv')
inp_v3 = pandas.read_csv('v3_rewrite_input.csv')

not_finished = []
done = 0
for i, r in inp_v2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r['s{}'.format(j)] not in finished:
            not_finished.append((r['url{}'.format(j)], r['s{}'.format(j)]))
        else:
            done += 1
        
for i, r in inp_v3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r['s{}'.format(j)] not in finished:
            not_finished.append((r['url{}'.format(j)], r['s{}'.format(j)]))
        else:
            done += 1

num = 5
with open('v23_left_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
    
    #fields.extend(["url11", "s11"])
    
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(not_finished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = not_finished[i + j][0]
            elem["s{}".format(j + 1)] = not_finished[i + j][1]
        writer.writerow(elem)

In [9]:
result = pandas.read_csv("input_file_yunkai.csv")
keys = set()
for i, r in result.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        keys.add(r["url{}".format(j)])

In [15]:
result = pandas.read_csv("v4_rewrite_input.csv")
#keys = set()
for i, r in result.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        if r["url{}".format(j)] in keys:
            keys.remove(r["url{}".format(j)])
unseen_tables = list(keys)

In [17]:
with open('v5_write_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    fields = ["url1"]
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for k in unseen_tables[:100]:
        writer.writerow({"url1": k})

In [57]:
with open('READY/prelim.json') as f:
    data = json.load(f)

pos = []
neg = []
pos_length = 0
neg_length = 0
for k in data:
    text = data[k]['text']
    label = data[k]['label']
    for t, l in zip(text, label):
        if l == 1:
            pos.append(t)
            pos_length += len(t.split())
        else:
            neg.append(t)
            neg_length += len(t.split())

print len(pos), len(neg)
print (pos_length + 0.0) / len(pos)
print (neg_length + 0.0) / len(neg)

9337 6791
11.989182821
12.0998380209


In [62]:
from collections import Counter

a = Counter()
a.update([1,2,3])
a.update([1,3,4])

print a

Counter({1: 2, 3: 2, 2: 1, 4: 1})


In [16]:
import pandas
import csv

r1 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3580679_batch_results.csv')
r2 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3584639_batch_results.csv')
r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]

r3 = pandas.concat([r1, r2])
finished = []

r1_inp = pandas.read_csv('v23_left_rewrite_input.csv')
r2_inp = pandas.read_csv('v4_rewrite_input.csv')
r3_inp = pandas.concat([r1_inp, r2_inp])

for i, r in r3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        finished.append(r["Input.s{}".format(j)])

print len(finished)
unfinished = []
done = 0
finished_old = []
for i,r in r3_inp.iterrows():
    if "url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        if r["s{}".format(j)]  not in finished:
            unfinished.append((r["url{}".format(j)], r["s{}".format(j)]))
        else:
            finished_old.append(r["s{}".format(j)])
            done += 1

print done
with open('v234_rest_rewrite_input.csv', 'w') as f:
    #json.dump(items, f, indent=2)
    num = 5
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(unfinished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = unfinished[i + j][0]
            elem["s{}".format(j + 1)] = unfinished[i + j][1]
            
        writer.writerow(elem)

10830
10892


In [24]:
r1 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3583922_batch_results.csv')
r2 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3579309_batch_results.csv')
r3 = pandas.read_csv('/Users/wenhuchen/Downloads/Batch_3579785_batch_results.csv')

r1 = r1[r1.AssignmentStatus=="Approved"]
r2 = r2[r2.AssignmentStatus=="Approved"]
r3 = r3[r3.AssignmentStatus=="Approved"]

r4 = pandas.concat([r1, r2, r3])
finished = []


for i, r in r4.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        finished.append(r["Input.s{}".format(j)])
print len(finished)

r1_inp = pandas.read_csv('v2_rewrite_input.csv')
r2_inp = pandas.read_csv('v3_rewrite_input.csv')
r3_inp = pandas.read_csv('v4_rewrite_input.csv')
r4_inp = pandas.concat([r1_inp, r2_inp, r3_inp])
unfinished = []
done = 0
for i,r in r4_inp.iterrows():
    if "url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        #if r["url{}".format(j)] in keys:
        if r["s{}".format(j)]  not in finished:
            unfinished.append((r["url{}".format(j)], r["s{}".format(j)]))
        else:
            done += 1

print done
with open('v234_rest_rewrite_fake_input.csv', 'w') as f:
    num = 5
    fields = []
    for j in range(1, num + 1):
        fields.append("url{}".format(j))
        fields.append("s{}".format(j))
        
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for i in range(0, len(unfinished) - num, num):
        elem = {}
        for j in range(num):
            elem["url{}".format(j + 1)] = unfinished[i + j][0]
            elem["s{}".format(j + 1)] = unfinished[i + j][1]
            
        writer.writerow(elem)

12385
12630


In [94]:
import re

def substitute(string):
    string = string.lower()
    
    string = string.replace(r"\n", "")
    string = string.replace(r"\\", "")
    string = string.replace(r"!$", "")
    string = string.replace(r"#", "")
    string = string.replace('–', '-')
    string = string.replace("√", "")
    string = string.replace(r'"', '')
    string = string.replace(r'\.0 ', '')
    string = string.replace(r'¹', '')
    
    string = string.replace(r"'s", " 's")
    string = string.replace(r"'re", " 're")
    string = string.replace(r"'nt", " not")
    string = string.replace(r'@', ' ')
    string = string.replace(r'`', '')
    string = string.replace(r'•', ' ')
    string = string.replace(r'·', ' ')
    string = string.replace(r'²', ' square')
    string = string.replace(r'\[', '')
    string = string.replace(r'\]', '')
    string = string.replace(r'\{', '')
    string = string.replace(r'\}', '')
    string = string.replace(r'\|', '')
    string = string.replace(r'\^', '')

    string = re.sub(r'\[.+\]', ' ', string)
    string = re.sub(r',([0-9]{3})(?![0-9])', r'\1', string)
    #string = re.sub(r'([0-9]{1-3}),([0-9]{3}),([0-9]{3})([^0-9])', r"\1\2\3\4", string)
    #string = re.sub(r'([0-9]{1-3}),([0-9]{3}),([0-9]{3}),([0-9]{3})([^0-9])', r"\1\2\3\4\5", string)
    #string = re.sub(r'([0-9]{1-3}),([0-9]{3}),([0-9]{3}),([0-9]{3}),([0-9]{3})([^0-9])', r"\1\2\3\4\5\6", string)
    
    string = re.sub(r'([^,]),', r"\1 ,", string)
    string = re.sub(r',([^,])', r", \1", string)
    string = re.sub(r'([^(])\(', r"\1 (", string)
    string = re.sub(r'\)([^)])', r") \1", string)
    string = re.sub(r'\.\)', r")", string)
    string = re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)-([0-9]+)=', r"\1+\2+\3+\4=", string)
    string = re.sub(r'([0-9]+)-([0-9]+)-([0-9]+)=', r"\1+\2+\3=", string)
    string = re.sub(r'([0-9]+)-([0-9]+)=', r"\1+\2=", string)
    string = re.sub(r"' +([0-9])", r"'\1", string)

    string = re.sub(r"([^ ])\+", r"\1 +", string)
    string = re.sub(r"\+([^ ])", r"+ \1", string)
    string = re.sub(r"([^ ])=", r"\1 =", string)
    string = re.sub(r"=([^ ])", r"= \1", string)

    string = re.sub(r"-([^- ])", r"- \1", string)
    string = re.sub(r"([^- ])-", r"\1 -", string)
    string = re.sub(r"-([^- ])", r"- \1", string)
    string = re.sub(r"/([^ ])", r"/ \1", string)
    string = re.sub(r"([^ ])/", r"\1 /", string)
    
    string = re.sub(r'([^0-9 ]):', r"\1 :", string)
    string = re.sub(r':([^0-9 ])', r": \1", string)
    string = re.sub(r'([0-9])pm', r"\1 pm", string)
    string = re.sub(r'([0-9])am', r"\1 am", string)
    string = re.sub(r'([0-9])rpm', r"\1 rpm", string)
    string = re.sub(r'([0-9])km', r"\1 km", string)
    string = re.sub(r'([0-9])cm', r"\1 cm", string)
    string = re.sub(r'([0-9])m', r"\1 m", string)
    string = re.sub(r'([0-9])mm', r"\1 mm", string)
    string = re.sub(r'([0-9])kg', r"\1 kg", string)
    string = re.sub(r'([0-9])g', r"\1 g", string)
    string = re.sub(r'([0-9])kw', r"\1 kw", string)
    string = re.sub(r'([0-9])kv', r"\1 kv", string)
    string = re.sub(r'([0-9])mph', r"\1 mph", string)
    #string = re.sub(r'([0-9])@', r"\1 @", string)
    #string = re.sub(r'@([0-9])', r"@ \1", string)
    string = re.sub(r'category : articles with hcards', r"", string)
    string = re.sub(r'category : articles with hcard', r"", string)
    string = re.sub(r'category : articles without hcard', r"", string)
    
    string = re.sub(r"\.+$", '', string)
    string = re.sub(r',+$', '', string)    
    string = re.sub(r'\s+', ' ', string)
    string = re.sub(r'^ ', '', string)
    string = re.sub(r' $', '', string)
    string = re.sub('70 - 76 - 68 - 214', '70 + 76 + 68 = 214', string)
    return string

#print substitute("fa\g##ga,/// she got(ff)ff...")
#print substitute('(5) fa:"12-12-13=31')

In [88]:
print re.sub(r',([0-9]{3})(?![0-9])', r'\1', '10,332,443,393')

10332443393


In [97]:
def blacklist(string):
    black = ['bye', "no chart for", "km (mi)", "lb·ft", "ft (m)", "kg (lb)", 
             "a report of report", "report was report", "is 'report'", "?"]
    for b in black:
        if b in string:
            return True
    return False

sys_trash = []

In [100]:
# encoding=utf8
import sys
import pandas

r1_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_0.csv")
r1_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_1.csv")

r1 = pandas.concat([r1_1, r1_2])
r1 = r1[r1.AssignmentStatus=="Approved"]

f = open('/tmp/output.txt', 'w')

index = 0
finished = {}
trash = []
for i,r in r1.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        if html_name in sys_trash:
            continue
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            trash.append(html_name)
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)
                    
r2_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_2.csv")
r2_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_3.csv")
r2_3 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/refine_4.csv")

r2 = pandas.concat([r2_1, r2_2, r2_3])
r2 = r2[r2.AssignmentStatus=="Approved"]

for i,r in r2.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        if html_name in sys_trash:
            continue        
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            trash.append(r['Input.url{}'.format(j)].split('/')[-1])
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [1]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(1)


r3_1 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_0.csv")
r3_2 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_1.csv")
r3_3 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_2.csv")
r3_4 = pandas.read_csv("/Users/wenhuchen/Downloads/harvest/fake_3.csv")

r3 = pandas.concat([r3_1, r3_2, r3_3, r3_4])
r3 = r3[r3.AssignmentStatus=="Approved"]

f = open('/tmp/output.txt', 'w')

index = 0
for i,r in r3.iterrows():
    if "Input.url10" in r:
        num = 10
    else:
        num = 5
    for j in range(1, num + 1):
        orig_input = r["Answer.d{}".format(j)]
        html_name = r['Input.url{}'.format(j)].split('/')[-1]
        if html_name in sys_trash:
            continue
        if isinstance(orig_input, str) and orig_input.lower() in ["na", "n/a", "no"]:
            trash.append(r['Input.url{}'.format(j)].split('/')[-1])
        if isinstance(orig_input, str) and len(orig_input.split(' ')) > 3 and not blacklist(orig_input):
            replaced_sent = substitute(orig_input)
            if 'hcard' not in replaced_sent:
                print >> f, replaced_sent
                index += 1
                if html_name not in finished:
                    finished[html_name] = [[replaced_sent], [0]]
                else:
                    finished[html_name][0].append(replaced_sent)
                    finished[html_name][1].append(0)

f.close()
print index
import json
with open("READY/training_all.json", 'w') as f:
    json.dump(finished, f, indent=2)

In [99]:
print trash
sys_trash = trash + ["2-11916083-12.html", "2-1236260-1.html"]

In [229]:
#re.sub("\s+", " ", "Cowdyshack      was the title that originally aired on February 2 , 2008")

'Cowdyshack was the title that originally aired on February 2 , 2008'

In [50]:
from bs4 import BeautifulSoup
import urllib2
import csv

url = open('WikiTableQuestions/wikidata/all_html/2-18178551-5.html').readline().strip()
soup = BeautifulSoup(url, "html.parser")

table = soup.findAll("table", {"class":"wikitable"})[0]
rows = table.findAll("tr")

with open('/tmp/editors.csv', "w") as f:
    writer = csv.writer(f)
    for row in rows:
        csv_row = []
        for cell in row.findAll(["td", "th"]):
            csv_row.append(cell.get_text())
        writer.writerow(csv_row)

In [95]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import csv
import os
from bs4 import BeautifulSoup

for filename in os.listdir('WikiTableQuestions/wikidata/all_html/'):
    if filename.endswith(".html"):
        url = open('WikiTableQuestions/wikidata/all_html/' + filename).readline().strip()
        soup = BeautifulSoup(url, "html.parser")
        table = soup.findAll("table", {"class":"wikitable"})
        if len(table) > 0:
            table = table[0]
            rows = table.findAll("tr")
            with open('WikiTableQuestions/wikidata/all_csv/' + filename + '.csv', 'w') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter='#')
                for row in rows:
                    csv_row = []
                    for cell in row.findAll(["td", "th"]):
                        print cell.get_text()
                        csv_row.append(substitute(cell.get_text()))
                        print substitute(cell.get_text())
                        print ""
                    spamwriter.writerow(csv_row)

KeyboardInterrupt: 

In [16]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

import json
import csv

with open('READY/training_all.json') as f:
    data = json.load(f)
    
files = data.keys()

num = 8
index = 0
with open('verify_inputs.csv', 'w') as f:
    fields = []
    for i in range(1, num + 1):
        fields.extend(['url{}'.format(i), 's{}'.format(i)])
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()

    buf = {}
    for f in files:
        entry = data[f]
        for s, l in zip(entry[0], entry[1]):
            cur = len(buf) // 2 + 1
            if cur > num:
                writer.writerow(buf)
                index += 1
                cur = 1
                buf = {}
            buf['url{}'.format(cur)] = "https://raw.githubusercontent.com/wenhuchen/Table-Fact-Checking/master/data/all_csv/" + f + ".csv"
            buf['s{}'.format(cur)] = s
            """
            if l == 1:
                buf['o{}'.format(cur)] = "Entailed"
            else:
                buf['o{}'.format(cur)] = "Contradictory"
            """
        if index > 100:
            break
    

In [32]:
import os

table_ids = set()
for f in os.listdir('data/all_csv/'):
    if f.endswith('csv'):
        table_id = f.split('-')[1]
        table_ids.add(table_id)

with open('tables.txt', 'w') as f:
    for t in table_ids:
        print >> f, t